Lets examine the links and scrape 


In [47]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


In [46]:
all_url = "https://www.allrecipes.com/recipes-a-z-6735880"
url = "https://www.allrecipes.com/smoked-jalapeno-poppers-recipe-7969057"

Block to scrape the actual recipe

In [48]:
start=url.rfind('-')+1
rkey=url[start:]

In [49]:
response=requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    #WHERE all recipe data is stored
    soup = soup.body.main.article
else:
    print("Failed to retrieve the web page.")



Build recipe dictionary

In [50]:
recipe ={}
recipe['id_recipe']=rkey

Grab the prep block info

In [51]:
rlabels = soup.find_all("div", {"class": "mntl-recipe-details__label"})
rinfo = soup.find_all("div", {"class": "mntl-recipe-details__value"})

label=[lab.text for lab in rlabels]
value=[val.text for val in rinfo]

recipe.update(dict(zip(label,value)))

In [52]:
display(recipe)

{'id_recipe': '7969057',
 'Prep Time:': '15 mins',
 'Cook Time:': ' 2 hrs',
 'Total Time:': ' 2 hrs 15 mins',
 'Servings:': '6 '}

Get Nutritional header data

In [53]:
#grab nutritional data
# Find the <th> element with the specified class
th_element = soup.find('thead', class_='mntl-nutrition-facts-label__table-head')
soup_head = th_element
# Check if the <th> element was found

# Find the <span> element within the <th> element
span_element = soup_head.find_all('span')
# Check if the <span> element was found

#2nd and 4th element are the data values
recipe[span_element[0].text.strip()]=span_element[1].text.strip()
recipe[span_element[2].text.strip()]=span_element[3].text.strip()


In [54]:
#grab nutritional data
# Find the <th> element with the specified class
td_element = soup.find('tbody', class_="mntl-nutrition-facts-label__table-body type--cat")
soup_body = td_element

#

# Find the <td> element
element = soup_body.find_all('tr')

for el in element:
    nlabel= el.find('span')
    if nlabel:
        nvalue=el.find_next('td')
        nvalue=nvalue.text.strip().splitlines()
        recipe[nvalue[0]]=nvalue[1]

soup.find('span', class_="mntl-nutrition-facts-label__table-body type--cat")


In [57]:
# Find the <div> element with the rating
rating_div = soup.find('div', class_='comp type--squirrel-bold mntl-recipe-review-bar__rating mntl-text-block')

# Find the <div> element with the rating count
count_div = soup.find('div', class_='comp type--squirrel mntl-recipe-review-bar__rating-count mntl-text-block')
try:
# Extract the rating and count from the <div> elements
    rating = rating_div.text.strip()
    count = count_div.text.strip().strip('()')

    recipe['Average_Review'] = rating
    recipe['Review_Count'] = count
except:
    pass

In [58]:
print(recipe)
rec_df = pd.DataFrame(recipe, index=[0])

{'id_recipe': '7969057', 'Prep Time:': '15 mins', 'Cook Time:': ' 2 hrs', 'Total Time:': ' 2 hrs 15 mins', 'Servings:': '6 ', 'Servings Per Recipe': '6', 'Calories': '189', 'Total Fat': '16g', 'Saturated Fat': '9g', 'Cholesterol': '43mg', 'Sodium': '474mg', 'Total Carbohydrate': '7g', 'Dietary Fiber': '2g', 'Total Sugars': '2g', 'Protein': '6g', 'Vitamin C': '3mg', 'Calcium': '99mg', 'Iron': '1mg', 'Potassium': '140mg'}


In [59]:
rec_df.head(
)

,id_recipe,Prep Time:,Cook Time:,Total Time:,Servings:,Servings Per Recipe,Calories,Total Fat,Saturated Fat,Cholesterol,Sodium,Total Carbohydrate,Dietary Fiber,Total Sugars,Protein,Vitamin C,Calcium,Iron,Potassium
0,7969057,15 mins,2 hrs,2 hrs 15 mins,6,6,189,16g,9g,43mg,474mg,7g,2g,2g,6g,3mg,99mg,1mg,140mg


Grab ingredients - this is a seperate DF


In [19]:
ul_element = soup.find('ul', class_='mntl-structured-ingredients__list')

grab ingredients

In [218]:


# Find the <ul> element with the specified class
ul_element = soup.find('ul', class_='mntl-structured-ingredients__list')

# Check if the <ul> element was found
if ul_element:
    # Find all <li> elements within the <ul> element
    li_elements = ul_element.find_all('li', class_='mntl-structured-ingredients__list-item')
    
    # Initialize lists to store extracted data
    quantities = []
    units = []
    ingredients = []
    
    # Iterate through the <li> elements and extract data
    for li in li_elements:
        span_quantity = li.find('span', {'data-ingredient-quantity': 'true'})
        span_unit = li.find('span', {'data-ingredient-unit': 'true'})
        span_ingredient = li.find('span', {'data-ingredient-name': 'true'})
        
        if span_quantity:
            quantities.append(span_quantity.text.strip())
        else:
            quantities.append('')
        
        if span_unit:
            units.append(span_unit.text.strip())
        else:
            units.append('')
        
        if span_ingredient:
            ingredients.append(span_ingredient.text.strip())
        else:
            ingredients.append('')
    
    # Print the extracted data
    temp_ing_df = pd.DataFrame({'id_recipe':rkey,'Quantity': quantities, 'Unit of Measure': units, 'Ingredient': ingredients})
    
else:
    print("The <ul> element with class 'mntl-structured-ingredients__list' was not found in the HTML.")

In [219]:
print(temp_ing_df)

  id_recipe Quantity Unit of Measure               Ingredient
0   7814346      1/4             cup        all-purpose flour
1   7814346      1/4        teaspoon                     salt
2   7814346      1/8        teaspoon      ground black pepper
3   7814346        1           large                      egg
4   7814346      1/4             cup      talian bread crumbs
5   7814346        2     tablespoons       panko bread crumbs
6   7814346        2     tablespoons   grated Parmesan cheese
7   7814346    1 (14      ounce) can  hearts of palm, drained
8   7814346                                     cooking spray


In [27]:
#grab steps data
# Find the <th> element with the specified class
steps = soup.find('ol', class_="comp mntl-sc-block-group--OL mntl-sc-block mntl-sc-block-startgroup")

#
step_desc=[]
step_count=[]
# Find the <td> element
element = steps.find_all('li')
c=1
for el in element:
    par = el.find_all('p')
    step_desc.append("".join(p.text.strip() for p in par))
    step_count.append(c)
    c += 1

temp_steps_df = pd.DataFrame({'id_recipe':rkey,'Step_Count': step_count, 'Step Description': step_desc})


Begin work on scraping all available links and categories

In [7]:
response=requests.get(all_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    #WHERE all recipe data is stored
    body = soup.body.main
else:
    print("Failed to retrieve the web page.")

Build a table to grab the hrefs and the name

In [8]:
a_elements = body.find_all('a', class_='link-list__link type--dog-bold type--dog-link')
href=[]
text =[]

# Iterate through the found <a> elements and print their text and href attributes
for a in a_elements:
    href.append(a.get('href'))
    text.append(a.text.strip())

df_ref = pd.DataFrame({'URL': href, 'Category': text})

In [11]:
df_ref['Category']=df_ref['Category'].str.replace(' ','_').str.lower()

In [59]:
cat_url=df_ref['URL'][1]

response=requests.get(cat_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    #WHERE all recipe data is stored
    body = soup.body.main
else:
    print("Failed to retrieve the web page.")

In [13]:
a_elements = body.find_all('a', class_='comp card--image-top mntl-card-list-items mntl-document-card mntl-card card card--no-image')
href=[]
rec_id=[]
# Iterate through the found <a> elements and print their text and href attributes
for a in a_elements:
    href.append(a.get('href'))
    rec_id.append(a.get('data-doc-id'))

In [61]:
a_elements = body.find_all('a', class_='comp mntl-card-list-items mntl-document-card mntl-card card card--no-image')

# Iterate through the found <a> elements and print their text and href attributes
for a in a_elements:
    href.append(a.get('href'))
    rec_id.append(a.get('data-doc-id'))


In [65]:
cat_df= pd.DataFrame({'id_recipe':rec_id,'url_recipe': href})

In [12]:
df=''
print(len(df))

0


In [24]:
url="https://www.allrecipes.com/recipe/273688/swedish-hallongrottor-cookies-raspberry-caves/"
x=url.find('.com/')+5
isrec=url[x:]
if ("recipe" in isrec and "gallery" not in isrec and "article" not in isrec):
    print(isrec)

recipe/273688/swedish-hallongrottor-cookies-raspberry-caves/
